## Task 2 - Named Entity Recognition

Use NLTK and/or Camel libraries to build a “Named Entity Extraction” Model that can be used to extract name of person, country, organization, events...etc. The model can be trained on training data and tested on different data.

The notebook is composed __two sections__:
1. __Testing the pre-trained CAMeL-Lab NER model.__
2. __Training a blank Spacy NER model on the dataset used by the CAMeL-Lab NER model ([ANERcorp](https://camel.abudhabi.nyu.edu/anercorp/)).__

In [1]:
import pandas as pd 
import emoji  # for removing emojis
import pyarabic.araby as araby  # for removing diacritics
import re
import pprint
import random
import spacy
from spacy.util import minibatch, compounding
from tqdm import tqdm
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.ner import NERecognizer  # Testing the CAMeL-Lab pre-trained AraBERT NER model

/home/tamer/anaconda3/envs/psut-nlp-course/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Loading one of the tweet files for testing purposes

In [2]:
df = pd.read_csv("data/file1.txt", sep="\t").drop("tweetID", axis=1)

In [3]:
df

,tweetText
0,"الاعدام لعامل مطعم قتل زميله طعناً في ""البيادر..."
1,#الأخبار ▪ تأجيل محاكمة 7 إرهابيين بسبب غياب ا...
2,@helale9999 عشآن أعطيتك وحده صميم صرت ترمي أعذ...
3,#النهدي ثمانية قتلى في تفجير انتحاري بسيارة مف...
4,البحرين: ضبط مطلوبين متورطين في التفجير بالعكر...
...,...
4995,«بوكو حرام »تخطف 40 شابا في شمال شرق نيجيريا -...
4996,قصيدة مهداة إلى الشيخ/ ضيف الله بن سمار كلمات/...
4997,العفو عند المقبرة!!
4998,#للمرة الأولى منذ 92 عاما.. تركيا تسمح ببناء ك...


#### Adding regular expressions from Task 1

* Removing URLs, Hashtags, Mentions, and repeating dots.
* Removing repeating characters than occur more than twice.
* Removing emojis.
* Removing Arabic Diacritics.

In [4]:
def clean(sentence):
    def remove_emojis(string):
        return emoji.get_emoji_regexp().sub(u"", string)
    
    def remove_diacritics(string):
        return araby.strip_diacritics(string)

    re_general_pattern = r"https?:\/\/.*[\r\n]*|#\w+|@\w+|\.{2,}"
    re_repeating_character_pattern = r"(\w)\1{2,}"

    # 1- Removing URLs, Hashtags, Mentions, and repeating dots
    sentence = re.sub(re_general_pattern, "", sentence)
    # 2- Removing repeating characters that occur more than twice
    sentence = re.sub(re_repeating_character_pattern, r"\1", sentence)
    # 3- Removing emojis
    sentence = remove_emojis(sentence)
    # 4- Removing Arabic Diacritics
    sentence = remove_diacritics(sentence)

    return sentence

#### Loading pprint for more legible prints of NER predictions

In [5]:
pp = pprint.PrettyPrinter(indent=4)

### 1- Using CAMeL-Lab Named Entity Recognition Model

Trying the CAMeL Tools [GitHub](https://github.com/CAMeL-Lab/camel_tools) __pre-trained CAMeLBERT MSA Named Entity Recognition model__ [docs](https://camel-tools.readthedocs.io/en/stable/api/ner.html) for the Arabic Language.


* Huggingface [page](https://huggingface.co/CAMeL-Lab/bert-base-arabic-camelbert-msa-ner) (contains information about the language model and the dataset used for NER)

    "The model was built by fine-tuning the CAMeLBERT Modern Standard Arabic (MSA) model. For the fine-tuning, the [ANERcorp](https://camel.abudhabi.nyu.edu/anercorp/) dataset was used. The fine-tuning procedure and the hyperparameters that were used could be found in the paper "The Interplay of Variant, Size, and Task Type in Arabic Pre-trained Language Models.""


* Needed to have the RUST [compiler](https://www.rust-lang.org/learn/get-started)
* Needed to do the ```camel_data -i all``` command to download all required data and models for CAMeL Tools.
* Guided Demo on CAMeL Tools [Colab](https://colab.research.google.com/drive/1Y3qCbD6Gw1KEw-lixQx1rI6WlyWnrnDS?usp=sharing#scrollTo=gwBGDtiY088_)



__Notes from the Colab Demo regarding their NER model__:

"For each token in an input sentence, NERecognizer outputs a label that indicates the type of named-entity. The system outputs one of the following labels for each token: 'B-LOC', 'B-ORG', 'B-PERS', 'B-MISC', 'I-LOC', 'I-ORG', 'I-PERS', 'I-MISC', 'O'. Named-entites can either be a LOC (location), ORG (organization), PERS (person), or MISC (miscallaneous).

Labels beginning with B indicate that their corresponding tokens are the begininging of a multi-word named-entity or is a single-token named-entity'. Those begining with I indicate that their corresponding tokens are continuations of a multi-word named-entity. Words that aren't named-entities are given the 'O' label."

#### 1.1- Testing the model 

In [6]:
camel_ner_model = NERecognizer.pretrained()

In [7]:
dir(camel_ner_model)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_align_predictions',
 'labels',
 'labels_map',
 'model',
 'predict',
 'predict_sentence',
 'pretrained',
 'tokenizer',
 'use_gpu']

In [8]:
camel_ner_model.use_gpu

True

In [9]:
# Testing example in the docs
sentence = clean('إمارة أبوظبي هي إحدى إمارات دولة الإمارات العربية المتحدة السبع .')
sentence

/home/tamer/anaconda3/envs/psut-nlp-course/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  This is separate from the ipykernel package so we can avoid doing imports until


'إمارة أبوظبي هي إحدى إمارات دولة الإمارات العربية المتحدة السبع .'

In [10]:
sentence = simple_word_tokenize(sentence)

In [11]:
sentence

['إمارة',
 'أبوظبي',
 'هي',
 'إحدى',
 'إمارات',
 'دولة',
 'الإمارات',
 'العربية',
 'المتحدة',
 'السبع',
 '.']

In [12]:
labels = camel_ner_model.predict_sentence(sentence)

In [13]:
# Print the list of token-label pairs
pp.pprint(list(zip(sentence, labels)))

[   ('إمارة', 'O'),
    ('أبوظبي', 'B-LOC'),
    ('هي', 'O'),
    ('إحدى', 'O'),
    ('إمارات', 'O'),
    ('دولة', 'O'),
    ('الإمارات', 'B-LOC'),
    ('العربية', 'I-LOC'),
    ('المتحدة', 'I-LOC'),
    ('السبع', 'O'),
    ('.', 'O')]


#### 1.2 Testing the model on the first 10 tweets in file1.txt

In [14]:
for index, tweet in df.iterrows():
    if index == 10:
        break
        
    sentence = clean(tweet[0])
    sentence = simple_word_tokenize(sentence)
    labels = camel_ner_model.predict_sentence(sentence)
    print("---------")
    print(f"Tweet {index}\n")
    print(f"Original Tweet: {tweet[0]}\n")
    print("Predictions:")
    pp.pprint(list(zip(sentence, labels)))

---------
Tweet 0

Original Tweet: الاعدام لعامل مطعم قتل زميله طعناً في "البيادر" أيدت محكمة التمييز الحكم الصادر عن محكمة الجنايات الكبرى والقاضي... http://t.co/H0txdjv3Kn

Predictions:
[   ('الاعدام', 'O'),
    ('لعامل', 'O'),
    ('مطعم', 'O'),
    ('قتل', 'O'),
    ('زميله', 'O'),
    ('طعنا', 'O'),
    ('في', 'O'),
    ('"', 'O'),
    ('البيادر', 'O'),
    ('"', 'O'),
    ('أيدت', 'O'),
    ('محكمة', 'O'),
    ('التمييز', 'O'),
    ('الحكم', 'O'),
    ('الصادر', 'O'),
    ('عن', 'O'),
    ('محكمة', 'O'),
    ('الجنايات', 'O'),
    ('الكبرى', 'O'),
    ('والقاضي', 'O')]
---------
Tweet 1

Original Tweet: #الأخبار ▪ تأجيل محاكمة 7 إرهابيين بسبب غياب الدفاع: أجلت محكمة الجنايات بالعاصمة إلى تاريخ لاحق محاكمة سبعة إ... http://t.co/GM4jmpAWbR

Predictions:
[   ('تأجيل', 'O'),
    ('محاكمة', 'O'),
    ('7', 'O'),
    ('إرهابيين', 'O'),
    ('بسبب', 'O'),
    ('غياب', 'O'),
    ('الدفاع', 'O'),
    (':', 'O'),
    ('أجلت', 'O'),
    ('محكمة', 'O'),
    ('الجنايات', 'O'),
    ('بالعاصمة'

/home/tamer/anaconda3/envs/psut-nlp-course/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  This is separate from the ipykernel package so we can avoid doing imports until


### 2- Using Spacy NER on Arabic Text (Training a blank model on the ANERcorp dataset)

List of languages supported by Spacy (__Arabic not included__) [link](https://spacy.io/usage/models#languages)

__Note__: Used this blog as a reference on how to prepare data for training the Spacy NER model pipline [link](https://www.machinelearningplus.com/nlp/training-custom-ner-model-in-spacy/)

* Data format for training a Spacy NER model: [link](https://www.machinelearningplus.com/nlp/training-custom-ner-model-in-spacy/#formatofthetrainingexamples)

```
# training data example
TRAIN_DATA = [
              ("Walmart is a leading e-commerce company", {"entities": [(0, 7, "ORG")]}),
              ("I reached Chennai yesterday.", {"entities": [(19, 28, "GPE")]}),
              ("I recently ordered a book from Amazon", {"entities": [(24,32, "ORG")]}),
              ("I was driving a BMW", {"entities": [(16,19, "PRODUCT")]}),
              ("I ordered this from ShopClues", {"entities": [(20,29, "ORG")]}),
              ("Fridge can be ordered in Amazon ", {"entities": [(0,6, "PRODUCT")]})
             ]
```

#### 2.1 Testing reading the [ANERcorp](https://camel.abudhabi.nyu.edu/anercorp/) dataset

__Note__: Used this GitHub repository as a reference for reading the data [link](https://github.com/HassanAzzam/Arabic-NER/blob/master/train.py)

In [15]:
f = open("data/ANERcorp-CamelLabSplits/ANERCorp_CamelLab_train.txt", 'r')
sents = f.read().split('\n. O\n')
f.close()

In [16]:
sents[0]

'فرانكفورت B-LOC\n(د O\nب O\nأ) O\nأعلن O\nاتحاد B-ORG\nصناعة I-ORG\nالسيارات I-ORG\nفي O\nألمانيا B-LOC\nامس O\nالاول O\nأن O\nشركات O\nصناعة O\nالسيارات O\nفي O\nألمانيا B-LOC\nتواجه O\nعاما O\nصعبا O\nفي O\nظل O\nركود O\nالسوق O\nالداخلية O\nوالصادرات O\nوهي O\nتسعي O\nلان O\nيبلغ O\nالانتاج O\nحوالي O\nخمسة O\nملايين O\nسيارة O\nفي O\nعام O\n2002 O'

In [17]:
sents[0].split("\n")

['فرانكفورت B-LOC',
 '(د O',
 'ب O',
 'أ) O',
 'أعلن O',
 'اتحاد B-ORG',
 'صناعة I-ORG',
 'السيارات I-ORG',
 'في O',
 'ألمانيا B-LOC',
 'امس O',
 'الاول O',
 'أن O',
 'شركات O',
 'صناعة O',
 'السيارات O',
 'في O',
 'ألمانيا B-LOC',
 'تواجه O',
 'عاما O',
 'صعبا O',
 'في O',
 'ظل O',
 'ركود O',
 'السوق O',
 'الداخلية O',
 'والصادرات O',
 'وهي O',
 'تسعي O',
 'لان O',
 'يبلغ O',
 'الانتاج O',
 'حوالي O',
 'خمسة O',
 'ملايين O',
 'سيارة O',
 'في O',
 'عام O',
 '2002 O']

In [18]:
sents[0].split("\n")[0].split(" ")

['فرانكفورت', 'B-LOC']

In [19]:
test = sents[0].split("\n")

In [20]:
test

['فرانكفورت B-LOC',
 '(د O',
 'ب O',
 'أ) O',
 'أعلن O',
 'اتحاد B-ORG',
 'صناعة I-ORG',
 'السيارات I-ORG',
 'في O',
 'ألمانيا B-LOC',
 'امس O',
 'الاول O',
 'أن O',
 'شركات O',
 'صناعة O',
 'السيارات O',
 'في O',
 'ألمانيا B-LOC',
 'تواجه O',
 'عاما O',
 'صعبا O',
 'في O',
 'ظل O',
 'ركود O',
 'السوق O',
 'الداخلية O',
 'والصادرات O',
 'وهي O',
 'تسعي O',
 'لان O',
 'يبلغ O',
 'الانتاج O',
 'حوالي O',
 'خمسة O',
 'ملايين O',
 'سيارة O',
 'في O',
 'عام O',
 '2002 O']

In [21]:
test[0]

'فرانكفورت B-LOC'

In [22]:
test[0].split(" ")

['فرانكفورت', 'B-LOC']

In [23]:
test[0].split(" ")[0]

'فرانكفورت'

In [24]:
len(test[0].split(" ")[0])

9

In [25]:
training_record = tuple()
training_sentence = []

start_index = 0
end_index = 0
entities_list = []

for item in test:
    split_item = item.split(" ")
    training_sentence.append(split_item[0])
    
    end_index += len(split_item[0])
    
    if split_item[1] != "O":  # "Other" label is ignored
        entities_list.append((start_index, end_index, split_item[1]))
    
    start_index += len(split_item[0]) + 1  # One extra for whitespace
    end_index += 1  # One extra for whitespace
        
training_sentence = " ".join(training_sentence)

In [26]:
training_sentence

'فرانكفورت (د ب أ) أعلن اتحاد صناعة السيارات في ألمانيا امس الاول أن شركات صناعة السيارات في ألمانيا تواجه عاما صعبا في ظل ركود السوق الداخلية والصادرات وهي تسعي لان يبلغ الانتاج حوالي خمسة ملايين سيارة في عام 2002'

In [27]:
entities_list

[(0, 9, 'B-LOC'),
 (23, 28, 'B-ORG'),
 (29, 34, 'I-ORG'),
 (35, 43, 'I-ORG'),
 (47, 54, 'B-LOC'),
 (92, 99, 'B-LOC')]

In [28]:
training_sentence[0: 9]

'فرانكفورت'

In [29]:
training_sentence[23: 28]

'اتحاد'

In [30]:
training_sentence[29: 34]

'صناعة'

In [31]:
training_sentence[35: 43]

'السيارات'

In [32]:
training_sentence[47: 54]

'ألمانيا'

In [33]:
training_sentence[92: 99]

'ألمانيا'

In [34]:
dictionary = {}

In [35]:
dictionary["entities"] = entities_list

In [36]:
dictionary

{'entities': [(0, 9, 'B-LOC'),
  (23, 28, 'B-ORG'),
  (29, 34, 'I-ORG'),
  (35, 43, 'I-ORG'),
  (47, 54, 'B-LOC'),
  (92, 99, 'B-LOC')]}

In [37]:
training_list = []
training_list.append(training_sentence)
training_list.append(dictionary)

training_tuple = tuple(training_list)

In [38]:
pprint.pprint(training_tuple)

('فرانكفورت (د ب أ) أعلن اتحاد صناعة السيارات في ألمانيا امس الاول أن شركات '
 'صناعة السيارات في ألمانيا تواجه عاما صعبا في ظل ركود السوق الداخلية '
 'والصادرات وهي تسعي لان يبلغ الانتاج حوالي خمسة ملايين سيارة في عام 2002',
 {'entities': [(0, 9, 'B-LOC'),
               (23, 28, 'B-ORG'),
               (29, 34, 'I-ORG'),
               (35, 43, 'I-ORG'),
               (47, 54, 'B-LOC'),
               (92, 99, 'B-LOC')]})


#### 2.2 Creating training dataset for Spacy NER blank model

In [39]:
TRAIN_DATA = []

In [40]:
for sent in tqdm(sents):
    training_list = []
    training_sentence = []
    entities_list = []
    entities_dictionary = {}
    
    start_index = 0
    end_index = 0
    
    sent_tokens = sent.split("\n")
    
    for sent_token in sent_tokens:
        split_items = sent_token.split(" ")
        
        training_sentence.append(split_items[0])

        end_index += len(split_items[0])

        if len(split_items) > 1:
            if split_items[1] != "O":  # "Other" label is ignored
                entities_list.append((start_index, end_index, split_items[1]))

        start_index += len(split_items[0]) + 1  # One extra for whitespace
        end_index += 1  # One extra for whitespace

    training_sentence = " ".join(training_sentence)
    entities_dictionary["entities"] = entities_list
    training_list.append(training_sentence)
    training_list.append(entities_dictionary)
    
    training_tuple = tuple(training_list)
    
    TRAIN_DATA.append(training_tuple)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3992/3992 [00:00<00:00, 46171.20it/s]


In [41]:
len(TRAIN_DATA)

3992

In [42]:
TRAIN_DATA[0]

('فرانكفورت (د ب أ) أعلن اتحاد صناعة السيارات في ألمانيا امس الاول أن شركات صناعة السيارات في ألمانيا تواجه عاما صعبا في ظل ركود السوق الداخلية والصادرات وهي تسعي لان يبلغ الانتاج حوالي خمسة ملايين سيارة في عام 2002',
 {'entities': [(0, 9, 'B-LOC'),
   (23, 28, 'B-ORG'),
   (29, 34, 'I-ORG'),
   (35, 43, 'I-ORG'),
   (47, 54, 'B-LOC'),
   (92, 99, 'B-LOC')]})

In [43]:
sents[1].split("\n")

['',
 'وقال O',
 'رئيس O',
 'الاتحاد O',
 'برند B-PERS',
 'جوتشولك I-PERS',
 'عند O',
 'إعلان O',
 'آخر O',
 'تقرير O',
 'سنوي O',
 'للاتحاد O',
 'إن O',
 'مستقبل O',
 'السوق O',
 'مازال O',
 'يفتقر O',
 'إلي O',
 'الخطوط O',
 'الواضحة O']

In [44]:
TRAIN_DATA[1]

(' وقال رئيس الاتحاد برند جوتشولك عند إعلان آخر تقرير سنوي للاتحاد إن مستقبل السوق مازال يفتقر إلي الخطوط الواضحة',
 {'entities': [(19, 23, 'B-PERS'), (24, 31, 'I-PERS')]})

In [45]:
TRAIN_DATA[1][0][19: 23]

'برند'

In [46]:
TRAIN_DATA[1][0][24: 31]

'جوتشولك'

In [47]:
sents[2].split("\n")

['',
 'وعلي O',
 'الرغم O',
 'من O',
 'أنه O',
 'قال O',
 'أنه O',
 'يتوقع O',
 'أن O',
 'تظل O',
 'صادرات O',
 'السيارات O',
 'عند O',
 'مستوي O',
 'مرتفع O',
 'هذا O',
 'العام O',
 'فإنه O',
 'يبدو O',
 'من O',
 'غير O',
 'المحتمل O',
 'أن O',
 'تصل O',
 'إلي O',
 'مستوي O',
 'سجل O',
 'نموها O',
 'عام O',
 '2001 O',
 'عندما O',
 'زادت O',
 'صادرات O',
 'سيارات O',
 'الركاب O',
 'بنسبة O',
 'ستة O',
 'في O',
 'المائة O',
 'لتصل O',
 'إلي O',
 '6.3 O',
 'مليون O',
 'سيارة O']

In [48]:
TRAIN_DATA[2]

(' وعلي الرغم من أنه قال أنه يتوقع أن تظل صادرات السيارات عند مستوي مرتفع هذا العام فإنه يبدو من غير المحتمل أن تصل إلي مستوي سجل نموها عام 2001 عندما زادت صادرات سيارات الركاب بنسبة ستة في المائة لتصل إلي 6.3 مليون سيارة',
 {'entities': []})

#### 2.3 Training Spacy NER model

Code taken from the [blogpost](https://www.machinelearningplus.com/nlp/training-custom-ner-model-in-spacy/#trainingthenermodel)

In [49]:
# Preparing blank model
nlp = spacy.blank("en")
nlp.add_pipe(nlp.create_pipe('ner'))
nlp.begin_training()

In [50]:
# Getting the pipeline component
ner = nlp.get_pipe("ner")

# Adding labels to the `ner`
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

In [51]:
# Disable pipeline components you dont need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [52]:
# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 10 epochs
  for iteration in tqdm(range(10)):

    # shuffling examples before every epoch
    random.shuffle(TRAIN_DATA)
    losses = {}
    
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
    
    for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                  )

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [09:02<00:00, 54.27s/it]


In [53]:
losses

{'ner': 2071.688857027133}

#### 2.4 Testing the Spacy model

In [54]:
# Testing the model
doc = nlp(clean("إمارة أبوظبي هي إحدى إمارات دولة الإمارات العربية المتحدة السبع ."))

print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('أبوظبي', 'B-LOC'), ('الإمارات', 'B-LOC'), ('العربية', 'I-LOC'), ('المتحدة', 'I-LOC')]


/home/tamer/anaconda3/envs/psut-nlp-course/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  This is separate from the ipykernel package so we can avoid doing imports until


In [55]:
# Testing on first 10 tweets from file1.txt
for index, tweet in df.iterrows():
    if index == 10:
        break
        
    sentence = clean(tweet[0])
    doc = nlp(sentence)
    labels = camel_ner_model.predict_sentence(sentence)
    print("---------")
    print(f"Tweet {index}\n")
    print(f"Original Tweet: {tweet[0]}\n")
    print("Predictions:")
    pp.pprint([(ent.text, ent.label_) for ent in doc.ents])

/home/tamer/anaconda3/envs/psut-nlp-course/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  This is separate from the ipykernel package so we can avoid doing imports until


---------
Tweet 0

Original Tweet: الاعدام لعامل مطعم قتل زميله طعناً في "البيادر" أيدت محكمة التمييز الحكم الصادر عن محكمة الجنايات الكبرى والقاضي... http://t.co/H0txdjv3Kn

Predictions:
[]
---------
Tweet 1

Original Tweet: #الأخبار ▪ تأجيل محاكمة 7 إرهابيين بسبب غياب الدفاع: أجلت محكمة الجنايات بالعاصمة إلى تاريخ لاحق محاكمة سبعة إ... http://t.co/GM4jmpAWbR

Predictions:
[]
---------
Tweet 2

Original Tweet: @helale9999 عشآن أعطيتك وحده صميم صرت ترمي أعذار ...حقق العالميةة و أرجع كلمني يَ الأياب الانتحاري

Predictions:
[('عشآن', 'B-PERS')]
---------
Tweet 3

Original Tweet: #النهدي ثمانية قتلى في تفجير انتحاري بسيارة مفخخة أمام معملين للغاز في ريف حمص - شبكة الصين http://t.co/r5zFEuzAPu

Predictions:
[('للغاز', 'B-ORG'), ('ريف', 'I-ORG'), ('الصين', 'B-LOC')]
---------
Tweet 4

Original Tweet: البحرين: ضبط مطلوبين متورطين في التفجير بالعكر الشرقي بقية الموضوع اضغط هنا http://t.co/t4A5bNrqyh

Predictions:
[('البحرين', 'B-LOC'), ('متورطين', 'B-LOC')]
---------
Tweet 5

Original Tweet: 

#### The training of the Spacy model would need some tuning